# Soil Testing Guide: Understanding and Tracking Your Land

Soil testing is the foundation of good land management. Whether you are applying for a grant,
planning amendments, or just trying to understand why one pasture outperforms another, soil
tests give you the numbers you need to make confident decisions.

This notebook explains what soil tests measure, how to read a lab report, how to calculate
amendment rates, and how to track your soil health over time with simple charts. Everything
is written in plain language for farmers, ranchers, and agronomists who want practical answers
from their soil data.

## How to Run This Notebook

1. Make sure you have Python and Jupyter installed. If you set up this project with `uv`, run:
   ```
   uv sync --extra data
   uv run jupyter notebook
   ```
   The `--extra data` flag installs matplotlib and numpy, which are used for charts.
2. Open this file (`soil-testing-guide.ipynb`) in the Jupyter interface.
3. Click into the first code cell and press **Shift + Enter** to run it. Repeat for each cell.
4. You can also use **Cell > Run All** from the menu to run everything at once.
5. Replace the sample data with your own lab results to generate personalized recommendations.

## Setup

We load the libraries used throughout this notebook. `pandas` organizes data into tables
and `matplotlib` creates charts.

In [ ]:
# Standard library

# Third-party
import pandas as pd

try:
    import matplotlib.pyplot as plt
    try:
        plt.style.use("seaborn-v0_8-whitegrid")
    except OSError:
        try:
            plt.style.use("ggplot")
        except OSError:
            pass  # Fall back to default style
    HAS_MATPLOTLIB = True
except ImportError:
    HAS_MATPLOTLIB = False
    print("Optional: install matplotlib for charts (uv sync --extra data)")

try:
    from tabulate import tabulate
    HAS_TABULATE = True
except ImportError:
    HAS_TABULATE = False

# Configuration
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 120)

print("Setup complete.")

: 

## What Soil Tests Measure

A standard soil test report contains a lot of numbers. Here is what the most important ones
mean in plain language.

In [ ]:
raw_test_descriptions = pd.DataFrame({
    "test": [
        "pH",
        "Organic Matter (OM%)",
        "Nitrogen (N)",
        "Phosphorus (P)",
        "Potassium (K)",
        "CEC",
        "Micronutrients",
        "Haney Score",
    ],
    "what_it_tells_you": [
        "How acidic or alkaline your soil is. Most crops do best between 6.0 and 7.0.",
        "The percentage of decomposed plant and animal material. Higher is better for water holding and fertility.",
        "Available nitrogen for plant growth. Often the most limiting nutrient in pastures.",
        "Available phosphorus. Important for root development and energy transfer in plants.",
        "Available potassium. Supports drought tolerance and disease resistance.",
        "Cation Exchange Capacity -- how well your soil holds onto nutrients. Higher CEC means less leaching.",
        "Zinc, manganese, iron, copper, boron. Needed in small amounts but deficiencies cause real problems.",
        "A biological soil health score (0-50+). Measures how much nutrition the soil biology can provide.",
    ],
})

print("Common Soil Test Parameters")
print("=" * 60)
for _, row in raw_test_descriptions.iterrows():
    print(f"\n{row['test']}")
    print(f"  {row['what_it_tells_you']}")

## Reading a Lab Report

Below is a sample lab report formatted as a table, similar to what you would receive from
a soil testing lab. The "rating" column tells you at a glance whether each value is a concern.

In [ ]:
raw_lab_report = pd.DataFrame({
    "test": [
        "pH", "Buffer pH", "Organic Matter", "Nitrate-N",
        "Phosphorus (Mehlich-3)", "Potassium", "Calcium",
        "Magnesium", "CEC", "Zinc", "Boron",
    ],
    "value": [5.8, 6.6, 2.3, 8, 18, 145, 1200, 180, 12.5, 1.2, 0.4],
    "unit": [
        "", "", "%", "ppm", "ppm", "ppm",
        "ppm", "ppm", "meq/100g", "ppm", "ppm",
    ],
    "rating": [
        "Low", "--", "Low", "Low", "Medium",
        "Medium", "Adequate", "Adequate", "Medium",
        "Adequate", "Low",
    ],
    "optimal_range": [
        "6.0 - 7.0", "--", "3.0 - 6.0%", "15 - 30 ppm",
        "25 - 50 ppm", "150 - 250 ppm", "1000 - 2000 ppm",
        "100 - 300 ppm", "10 - 25 meq/100g", "1.0 - 5.0 ppm",
        "0.5 - 2.0 ppm",
    ],
})

print("Sample Lab Report -- North Pasture, Fall 2025")
print("=" * 60)
if HAS_TABULATE:
    print(tabulate(raw_lab_report, headers="keys", tablefmt="grid", showindex=False))
else:
    print(raw_lab_report.to_string(index=False))

### Interpreting This Report

- **pH 5.8 (Low):** Slightly too acidic for most forage grasses. Lime application recommended.
- **Organic Matter 2.3% (Low):** Below the 3%+ target for productive pastures. Building OM
  through cover crops, compost, or improved grazing management is a long-term priority.
- **Nitrate-N 8 ppm (Low):** Nitrogen is limiting. This is common in low-OM soils because
  most available nitrogen comes from organic matter decomposition.
- **Phosphorus 18 ppm (Medium):** Not deficient, but adding P through manure or fertilizer
  could help root establishment if you are overseeding.
- **Boron 0.4 ppm (Low):** Boron deficiency can limit legume establishment. A small
  application of borax may help if you are planting clover.

## Soil Health Benchmarks

Use the table below to quickly rate your soil test results. These benchmarks apply broadly
to pastures and cropland in the eastern United States. Your local Extension office can
provide region-specific targets.

In [ ]:
raw_benchmarks = pd.DataFrame({
    "indicator": [
        "pH",
        "Organic Matter (%)",
        "Aggregate Stability (%)",
        "Infiltration Rate (in/hr)",
        "Earthworm Count (per sq ft)",
    ],
    "needs_work": [
        "< 5.5 or > 7.5",
        "< 2.0",
        "< 20",
        "< 0.5",
        "< 3",
    ],
    "acceptable": [
        "5.5 - 6.0",
        "2.0 - 3.0",
        "20 - 40",
        "0.5 - 1.0",
        "3 - 5",
    ],
    "good": [
        "6.0 - 6.8",
        "3.0 - 4.5",
        "40 - 60",
        "1.0 - 2.0",
        "5 - 10",
    ],
    "excellent": [
        "6.2 - 6.8",
        "> 4.5",
        "> 60",
        "> 2.0",
        "> 10",
    ],
})

print("Soil Health Benchmarks")
print("=" * 70)
if HAS_TABULATE:
    print(tabulate(raw_benchmarks, headers="keys", tablefmt="grid", showindex=False))
else:
    print(raw_benchmarks.to_string(index=False))

## Calculating Amendment Rates

Once you know what your soil needs, the next question is: how much should you apply?
Below we walk through the two most common calculations step by step.

### Lime Calculation

The amount of lime you need depends on your current pH, your buffer pH, and your target pH.
The buffer pH tells the lab how resistant your soil is to pH change -- a lower buffer pH
means you need more lime.

In [ ]:
# --- Lime calculation ---
# These values come from your lab report
current_ph = 5.8
buffer_ph = 6.6
target_ph = 6.5

# SMP buffer method: approximate tons of lime per acre
# This simplified formula works for most eastern US soils
lime_tons_per_acre = (target_ph - buffer_ph) * (-1.0) * (12.0 / (7.0 - buffer_ph))
lime_tons_per_acre = max(0, round(lime_tons_per_acre, 1))

print("Lime Calculation")
print("-" * 40)
print(f"Current pH:        {current_ph}")
print(f"Buffer pH:         {buffer_ph}")
print(f"Target pH:         {target_ph}")
print("")
print(f"Recommended lime:  {lime_tons_per_acre} tons/acre")
print("")
print("Note: This is agricultural-grade limestone (ECCE 60-70%).")
print("If using pelletized lime, the rate may differ. Ask your")
print("supplier for the ECCE rating to adjust.")

### Nitrogen-Phosphorus-Potassium (N-P-K) Fertilizer Rates

The calculation below estimates how much of a common fertilizer blend you need to meet
a target nutrient rate. We use the example of applying 60 lbs of actual nitrogen per acre.

In [ ]:
# --- N-P-K fertilizer rate ---
# Example: 19-19-19 triple blend
fertilizer_n_pct = 19  # percent N in the bag
fertilizer_p_pct = 19  # percent P2O5
fertilizer_k_pct = 19  # percent K2O

target_n_lbs_per_acre = 60  # desired actual N

# How many lbs of product to apply
product_lbs_per_acre = target_n_lbs_per_acre / (fertilizer_n_pct / 100)

# What P and K you get along with it
actual_p_lbs = product_lbs_per_acre * (fertilizer_p_pct / 100)
actual_k_lbs = product_lbs_per_acre * (fertilizer_k_pct / 100)

print("Fertilizer Rate Calculation")
print("-" * 40)
print(f"Fertilizer:         {fertilizer_n_pct}-{fertilizer_p_pct}-{fertilizer_k_pct}")
print(f"Target N:           {target_n_lbs_per_acre} lbs actual N/acre")
print("")
print(f"Product to apply:   {product_lbs_per_acre:.0f} lbs/acre")
print(f"Actual N applied:   {target_n_lbs_per_acre} lbs/acre")
print(f"Actual P2O5:        {actual_p_lbs:.0f} lbs/acre")
print(f"Actual K2O:         {actual_k_lbs:.0f} lbs/acre")
print("")
print(f"Cost estimate at $0.55/lb product: ${product_lbs_per_acre * 0.55:,.2f}/acre")

## Tracking Soil Health Over Time

One soil test is a snapshot. The real value comes from testing the same fields year after year
and watching the trends. Below we use sample data from a farm that has been testing annually
for five years to show how organic matter and pH respond to management changes.

In [ ]:
# Multi-year soil test data for two fields
raw_soil_history = pd.DataFrame({
    "year": [2021, 2022, 2023, 2024, 2025] * 2,
    "field": (
        ["North Pasture"] * 5 + ["South Pasture"] * 5
    ),
    "organic_matter_pct": [
        2.1, 2.3, 2.5, 2.9, 3.2,   # North - under rotational grazing
        2.0, 2.0, 1.9, 2.1, 2.1,   # South - continuous grazing (control)
    ],
    "ph": [
        5.5, 5.8, 6.0, 6.2, 6.3,   # North - limed in 2021
        5.6, 5.5, 5.4, 5.5, 5.4,   # South - no lime
    ],
})

print("Multi-Year Soil Test History")
print("=" * 50)
if HAS_TABULATE:
    print(tabulate(raw_soil_history, headers="keys", tablefmt="grid", showindex=False))
else:
    print(raw_soil_history.to_string(index=False))

### Organic Matter Trend Chart

This chart compares organic matter trends between the two fields. The North Pasture has been
under rotational grazing since 2021, while the South Pasture remained under continuous grazing.

In [ ]:
if HAS_MATPLOTLIB:
    fig, ax = plt.subplots(figsize=(9, 5))

    for field_name, group in raw_soil_history.groupby("field"):
        ax.plot(
            group["year"],
            group["organic_matter_pct"],
            marker="o",
            linewidth=2,
            label=field_name,
        )

    ax.set_title("Soil Organic Matter Over Time", fontsize=14)
    ax.set_xlabel("Year")
    ax.set_ylabel("Organic Matter (%)")
    ax.set_ylim(1.5, 4.0)
    ax.legend()
    ax.axhline(y=3.0, color="green", linestyle="--", alpha=0.5, label="Target (3%)")
    ax.legend()
    plt.tight_layout()
    plt.show()
else:
    print("Install matplotlib to see this chart: uv sync --extra data")

### pH Trend Chart

This chart shows how the North Pasture pH responded to lime application in 2021, compared
to the South Pasture which received no lime.

In [ ]:
if HAS_MATPLOTLIB:
    fig, ax = plt.subplots(figsize=(9, 5))

    for field_name, group in raw_soil_history.groupby("field"):
        ax.plot(
            group["year"],
            group["ph"],
            marker="s",
            linewidth=2,
            label=field_name,
        )

    ax.set_title("Soil pH Over Time", fontsize=14)
    ax.set_xlabel("Year")
    ax.set_ylabel("pH")
    ax.set_ylim(5.0, 7.0)
    ax.axhspan(6.0, 6.8, alpha=0.1, color="green", label="Optimal range")
    ax.legend()
    plt.tight_layout()
    plt.show()
else:
    print("Install matplotlib to see this chart: uv sync --extra data")

## When and How to Collect Samples

Good data starts with good sampling. Follow this protocol to get consistent, reliable results.

### Timing
- **Best time:** Fall, after harvest or the end of the grazing season.
- **Consistency matters:** Test at the same time each year so results are comparable.
- **Avoid:** Right after applying fertilizer, lime, or manure (wait at least 6-8 weeks).

### Depth
- **Standard:** 0-6 inches for most routine tests (pH, nutrients, OM).
- **Deep profile:** 6-12 inches if testing for compaction layers or deep nutrient placement.

### Sampling Pattern
- Walk a **zigzag or W pattern** across the field.
- Collect **15-20 cores per sample** and mix them together in a clean bucket.
- Avoid unusual spots: fence lines, old manure piles, wet areas, and field borders.

### Number of Samples
- One composite sample per **10-20 acres** of uniform soil.
- If a field has obviously different zones (hilltop vs. bottom, sandy vs. clay), sample each zone separately.

## Connecting Soil Data to Grant Applications

Your soil test results are some of the strongest evidence you can include in a grant proposal.
Here is how to use them effectively.

1. **Establish a baseline.** Test every field before you start your project. This is your
   "before" picture.
2. **Set measurable targets.** Instead of writing "improve soil health," write "increase
   organic matter from 2.1% to 3.0% over 3 years."
3. **Show the trend.** If you have been testing for multiple years, include a chart like
   the ones above. Reviewers love to see data.
4. **Connect the dots.** Explain why your management changes will lead to the outcomes
   your soil data suggests are needed.

For a complete guide to building a grant proposal around your soil data, see the
[Grant Writing for Farms](grant-writing-for-farms.ipynb) notebook in this same folder.

## Conclusion

Soil testing does not have to be complicated. A basic test costs $15-30 per sample and gives
you the information you need to make smart decisions about lime, fertilizer, and management.
The most important thing is to start testing and keep testing consistently.

### Next Steps Checklist

- [ ] Identify the fields or pastures you want to test
- [ ] Collect soil samples following the protocol in this notebook (15-20 cores per sample, 0-6 inch depth)
- [ ] Send samples to a reputable lab (contact your local Extension office for recommendations)
- [ ] Enter your results into this notebook to calculate amendment rates
- [ ] If pH is below 6.0, get a lime recommendation and apply before the next growing season
- [ ] Set up an annual testing schedule -- same fields, same time each year
- [ ] Keep a simple spreadsheet or notebook tracking your results year over year
- [ ] Use your soil data in grant applications (see the [Grant Writing for Farms](grant-writing-for-farms.ipynb) notebook)
- [ ] Share your results with your NRCS conservationist or Extension agent for personalized advice